CNN Kullanarak Görüntü Sınıflandırma

Bu projede, görüntü sınıflandırmak için bir Konvolüsyonel Sinir Ağı (CNN) modeli oluşturacağız. Modeli oluşturmak için TensorFlow ve Keras kütüphanelerini kullanacağız. Veri seti eğitim ve test olarak ikiye ayrılacak ve görüntü ön işleme teknikleriyle modelin doğruluğunu artırmayı hedefleyeceğiz.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

from PIL import Image
import os
import warnings


numpy ve pandas: Veri ve matris işlemleri için kullanılır.

matplotlib ve seaborn: Veri kümesini ve model performans metriklerini görselleştirmek için kullanılır.

train_test_split: Veriyi eğitim ve test setlerine ayırmak için kullanılır.

confusion_matrix ve classification_report: Modelin sınıflandırma performansını değerlendirmek için kullanılır.

Sequential, Dense, Dropout, Flatten, Conv2D, MaxPooling2D: CNN modelini oluşturmak için gerekli katmanlar.

ImageDataGenerator: Modeli eğitirken gerçek zamanlı veri artırımı sağlar.

LabelEncoder ve to_categorical: Kategorik etiketleri sayısal verilere dönüştürmek için kullanılır.

PIL kütüphanesi: Görüntü işlemleri için kullanılır.

In [ ]:
# Uyarıları yok say
warnings.filterwarnings('ignore')


Bazı gereksiz uyarıları yok sayarak çıktının daha temiz olmasını sağlayacağız.

In [ ]:
# Kaggle veri seti yolu
fish_dir = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'


Veri setinin dosya yolunu tanımlıyoruz. Veri seti Kaggle ortamında bulunduğundan, dosya yolunu belirtmemiz lazım.

In [ ]:
# Etiket ve yol listelerini tanımla
labels = []
paths = []


Bu adımda, etiketler (hangi tür balık olduğu) ve resim yollarını depolamak için iki liste oluşturuyoruz.

In [ ]:
# Klasörlerde gezinip her resmin yolunu ve etiketini kaydet
for dir_name, _, filenames in os.walk(fish_dir):
    for filename in filenames:
        if filename.endswith('.png') and 'GT' not in dir_name:
            labels.append(os.path.split(dir_name)[-1])
            paths.append(os.path.join(dir_name, filename))


Bu adımda, veri seti klasörlerinin içinde gezinerek her resmin yolunu ve etiketini topluyoruz. Her balık türü ayrı bir klasörde olduğu için, klasör ismi etiketi temsil ediyor.

os.walk: Belirtilen dizinde tüm alt dizinleri ve dosyaları döner.

png uzantısına sahip dosyaları alırız, çünkü veri setimizdeki resimler bu formatta.

'GT': 'GT' içeren klasörleri hariç tutuyoruz (genellikle bu tür klasörler "ground truth" içindir).

In [ ]:
# Pandas DataFrame'e dönüştür
data = pd.DataFrame({'path': paths, 'label': labels})


Topladığımız yolları ve etiketleri bir DataFrame'e dönüştürüyoruz. Bu, veriyi organize bir formatta tutmamızı sağlar.

path: Görüntü dosyalarının yolları.
label: Görüntülerin etiketleri (balık türleri).

In [ ]:
# Resimleri yükleme fonksiyonu
def load_images(data, img_size=(128, 128)):
    images = []
    for path in data['path']:
        img = Image.open(path)
        img = img.resize(img_size)
        img_array = np.array(img) / 255.0
        if len(img_array.shape) == 2:  # Eğer grayscale ise
            img_array = np.stack([img_array]*3, axis=-1)
        images.append(img_array)
    return np.array(images)


Bu adımda, her resmin yolunu alıp resimleri belirli bir boyutta yeniden boyutlandıran ve normalize eden bir fonksiyon tanımlıyoruz. Eğer görüntüler gri tonlamalıysa, bunları 3 kanallı RGB formatına dönüştürüyoruz.

img_size: Resimlerin yeniden boyutlandırılacağı hedef boyut (örneğin 128x128 piksel).

img_array / 255.0: Her bir pikseli [0, 1] aralığına getiririz, böylece model daha iyi öğrenir.

In [ ]:
# Görselleri yükle (Bellek problemi varsa ImageDataGenerator kullanın)
X = load_images(data)
y = data['label']


Resimleri yüklemek için daha önce tanımladığımız load_images fonksiyonunu çağırıyoruz

X: Resim verilerinin numpy dizisi.

y: Etiketler.

In [ ]:
# Etiketleri sayısal değerlere dönüştür
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)


Makine öğrenmesi algoritmalarında genellikle etiketler sayısal olarak ifade edilir. Bu adımda, etiketlerimizi sayısal değerlere dönüştürüyoruz ve sonrasında bu değerleri one-hot encoding (tek-sıcak kodlama) yaparak her sınıfı ayrı bir vektör olarak ifade ediyoruz.

LabelEncoder: Etiketleri sayısal değerlere dönüştürür.

to_categorical: Sayısal etiketleri one-hot encoding formuna dönüştürür.

In [ ]:
# Train-test verisini ayır
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)


Veri setimizi %80 eğitim, %20 test olacak şekilde ayırıyoruz.

train_test_split: Veri setini eğitim ve test olarak böler. test_size=0.2 parametresi, verinin %20'sinin test setine ayrılmasını sağlar.




In [ ]:
# Veri artırıcı kullanarak veri yükleme ve augmentasyon (alternatif yöntem)
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
                             horizontal_flip=True, fill_mode='nearest')

datagen.fit(X_train)


Veri artırıcı kullanarak görüntüler üzerinde çeşitli dönüşümler yapıyoruz. Bu sayede modelin genelleme yeteneği artıyor.

rotation_range: Görüntüleri belirli bir aralıkta döndürür.

width_shift_range ve height_shift_range: Görüntüleri yatay ve dikey olarak kaydırır.

horizontal_flip: Görüntüleri yatay olarak çevirir.

In [ ]:
# Modelin tanımlanması
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))


CNN modeli tanımlanıyor. İki konvolüsyon katmanı ve ardından havuzlama katmanları, ardından tam bağlantılı katmanlar ekleniyor.

Conv2D: Konvolüsyonel katman, özellikleri öğrenir.

MaxPooling2D: Boyutları küçültür ve en belirgin özellikleri öne çıkarır.

Dense: Tam bağlantılı katman, sınıflandırma işlemi yapar.

Dropout: Aşırı öğrenmeyi engellemek için belirli nöronları rastgele sıfırlar.

softmax: Çıkış katmanında, sınıflandırma için kullanılır.



In [ ]:
# Modelin derlenmesi
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Modeli adam optimizasyon algoritması ve kategorik çapraz entropi kaybı fonksiyonu ile derliyoruz.

adam: Optimizasyon algoritması, ağırlıkları günceller.

categorical_crossentropy: Çok sınıflı sınıflandırma problemlerinde kullanılan kayıp fonksiyonu.



In [ ]:
# Modeli eğitme (data generator kullanarak)
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_test, y_test), epochs=20)


Modeli veri artırıcı kullanarak eğitiyoruz ve doğrulama seti ile performansını değerlendiriyoruz.

datagen.flow: Veri artırıcı ile eğitim verilerini besler.

epochs: Modelin kaç defa eğitim seti üzerinde eğitileceğini belirtir.

validation_data: Eğitim sırasında doğrulama seti ile modelin performansını izler.

In [ ]:
# Modeli değerlendirme
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)


Test seti üzerinde tahminler yaparak modelin performansını değerlendiriyoruz.

predict: Test seti üzerinde tahmin yapar.

np.argmax: One-hot encoded vektörleri sınıf etiketlerine dönüştürür.

In [ ]:
# Confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.show()


Modelin sınıflandırma başarısını görselleştirmek için karışıklık matrisi oluşturuyoruz.

confusion_matrix: Modelin hangi sınıfları doğru veya yanlış sınıflandırdığını gösterir.

sns.heatmap: Matrisin görselleştirilmesi.

In [ ]:
# Classification report
print(classification_report(y_true, y_pred_classes, target_names=label_encoder.classes_))


Her sınıf için doğruluk, geri çağırma, F1 skoru gibi metrikleri içeren bir rapor oluşturuyoruz.



In [ ]:
# Accuracy grafiği
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


Modelin eğitim ve doğrulama sırasında elde ettiği doğruluk oranlarını görselleştiriyoruz.



In [ ]:
# Kayıp fonksiyonu grafiği
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


Eğitim ve doğrulama sırasında modelin kayıp fonksiyon değerlerini görselleştiriyoruz.



In [ ]:
# Hiperparametre optimizasyonu için dropout oranı ve optimizer değişiklikleri
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))


Modelde kullanılan dropout oranını ve optimizasyon algoritmasını değiştirerek modelin daha iyi performans göstermesini sağlamaya çalışıyoruz.

Dropout(0.4): Dropout oranını artırarak daha fazla nöron sıfırlıyoruz.

rmsprop: Optimizasyon algoritmasını rmsprop olarak değiştiriyoruz.

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_test, y_test), epochs=20)


Yeni dropout oranı ve optimizasyon algoritması ile modeli tekrar eğitiyoruz.

rmsprop: Daha iyi bir öğrenme hızına sahip alternatif bir optimizasyon algoritmasıdır.